In [104]:
import pandas as pd
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
import numpy as np

In [105]:
# conectarse a bbdd mysql
with open ('password.txt') as f:
   pwd=str(f.readline()).strip()

conexion = pymysql.connect(host='localhost',
                           database = 'henry_pi',
                           user= 'root',
                           password=pwd)

cursor = conexion.cursor()

In [106]:
# se le pasa el nombre de una tabla contenida en una base de datos a la cual se esta conectado
# devuelve un dataframe
def extraer_datos(tabla:str):
    cursor.execute("SELECT * FROM {}".format(tabla))
    rows = cursor.fetchall()
    headers = [column[0] for column in cursor.description]
    df= pd.DataFrame(rows,columns=headers)
    return df


In [107]:
# extraer datos de bbdd
historico_velocidad_nacional=extraer_datos('historico_velocidad_nacional')
ingresos_trimestre=extraer_datos('ingresos_trimestre')
internet_accesos_velocidad_provincias=extraer_datos('internet_accesos_velocidad_provincias')
internet_penetracion_hogares=extraer_datos('internet_penetracion_hogares')
internet_tipo_nacional=extraer_datos('internet_tipo_nacional')
internet_tipo_provincias=extraer_datos('internet_tipo_provincias')
localidades_servicios=extraer_datos('localidades_servicios')

In [108]:
# unir los dataframes segun la estructura de los datos
nacionales = [historico_velocidad_nacional,ingresos_trimestre,internet_tipo_nacional]
provinciales = [internet_accesos_velocidad_provincias,internet_penetracion_hogares,internet_tipo_provincias]
locales=[localidades_servicios]

In [109]:
# deshacerse de algunas columnas innecesarias
historico_velocidad_nacional.drop(columns='trimestre_d',inplace=True)
ingresos_trimestre.drop(columns='periodo',inplace=True)

In [110]:
df_nacionales = reduce(lambda left,right:pd.merge(left,right, on=['año','trimestre'],how='outer'),nacionales) # uso reduce para hacer un join de los dataframes

In [111]:
internet_tipo_provincias['trimestre']=internet_tipo_provincias.trimestre.astype(int)
internet_accesos_velocidad_provincias.rename(columns={'otros':'otras_velocidades'},inplace=True)
#internet_tipo_provincias.drop(columns='total',inplace=True)

In [112]:
df_provinciales = reduce(lambda left,right:pd.merge(left,right,on=['año','trimestre','provincia'],how='outer'),provinciales)

In [113]:
df_locales = locales[0]

In [114]:
# latitud y longitud a  float
df_locales['latitud']=df_locales['latitud'].str.replace('..','.')
df_locales['latitud']=df_locales['latitud'].astype(float)


In [115]:
df_locales['longitud']=df_locales['longitud'].str.replace('..','.')
df_locales['longitud']=df_locales['longitud'].astype(float)
df_locales.rename(columns={'latitud':'latitude','longitud':'longitude'})

,provincia,partido,localidad,poblacion,adsl,cable_modem,dial_up,fibra_opt,satelital,wireless,telefonia_fija,3g,4g,link,latitude,longitude
0,BUENOS AIRES,Bahía Blanca,Villa Bordeau,0,NO,NO,NO,NO,NO,NO,NO,SI,SI,6056020,-38.647261,-62.320022
1,BUENOS AIRES,Bahía Blanca,Villa Espora,0,NO,NO,NO,NO,NO,NO,NO,NO,NO,6056020,-38.776507,-62.185183
2,BUENOS AIRES,Balcarce,Balcarce,38376,SI,SI,SI,SI,NO,SI,SI,SI,SI,6063010,-37.848278,-58.255167
3,BUENOS AIRES,Balcarce,Los Pinos,337,NO,NO,NO,NO,NO,SI,SI,NO,NO,6063020,-37.941206,-58.322592
4,BUENOS AIRES,Balcarce,Napaleofú,374,NO,NO,NO,SI,NO,SI,SI,SI,SI,6063030,-37.625498,-58.746186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307,CHACO,Quitilipi,Quitilipi,24517,SI,NO,NO,SI,NO,SI,SI,SI,SI,22133010,-26.873207,-60.218524
4308,CHACO,Quitilipi,Villa El Palmar,879,NO,NO,NO,NO,NO,SI,NO,NO,SI,22133020,-26.455119,-60.164617
4309,CHACO,San Fernando,Barranqueras,54698,SI,SI,NO,SI,NO,SI,SI,SI,SI,22140010,-27.487774,-58.932742
4310,CHACO,San Fernando,Basail,1929,SI,SI,NO,NO,SI,SI,SI,SI,SI,22140020,-27.886866,-59.279100


In [116]:
df_locales.loc[df_locales['longitud']<-100,'longitud']=df_locales.longitud/1000
df_locales.loc[df_locales['latitud']<-100,'latitud']=df_locales.latitud/1000

In [117]:
# quitar caracteres especiales de provincias para georreferencias
mapping = {'á': 'a',
           'é': 'e',
           'í': 'i',
           'ó': 'o',
           'ú': 'u',
           'ñ': 'n',
           'D': 'd'}

df_provinciales.provincia.replace(mapping, regex=True,inplace=True)


In [118]:
df_provinciales['provincia'].unique()
# Santiago del Estero
# Santiago Del Estero

array(['Buenos Aires', 'Capital Federal', 'Catamarca', 'Chaco', 'Chubut',
       'Cordoba', 'Corrientes', 'Entre Rios', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquen',
       'Rio Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'Santiago del Estero', 'Tierra del Fuego', 'Tucuman'],
      dtype=object)

In [119]:
df_provinciales.head()

,año,trimestre,provincia,hasta_512kbps,512_kbps_1mbps,1mbps_6mbps,6mbps_10mbps,10mbps_20mbps,20mbps_30mbps,30mbps_x,otras_velocidades,total_x,accesos_100_hogares,banda_ancha_fija,dial_up,total_y
0,2022,3,Buenos Aires,29985.0,27709.0,290315.0,297915.00,267044.0,124.19,3618690.00,65821.0,4721670.0,78.11,4715469,6199,4721668
1,2022,3,Capital Federal,517.0,5742.0,34371.0,67829.00,51946.0,28692.00,1253100.00,105477.0,1547680.0,122.28,1545534,2145,1547679
2,2022,3,Catamarca,71.0,384.0,3107.0,5389.00,5099.0,3737.00,50298.00,2208.0,70293.0,65.33,70292,1,70293
3,2022,3,Chaco,461.0,987.0,16782.0,18938.00,8049.0,15828.00,79.39,3711.0,144146.0,43.86,144141,5,144146
4,2022,3,Chubut,109.0,1444.0,45707.0,30.94,34682.0,15309.00,17563.00,20024.0,165778.0,84.38,164874,904,165778


In [120]:
# corregir errores en los valores (venian con puntos mal colocados)
for i in df_provinciales:
    if (df_provinciales[i].dtype==float) & (i!='accesos_100_hogares'):
        floats=df_provinciales.loc[df_provinciales[i]%1 != 0,i]
        df_provinciales.loc[df_provinciales[i]%1 != 0,i]=floats*1000


In [121]:
df_provinciales[df_provinciales['provincia']=='Chubut']

,año,trimestre,provincia,hasta_512kbps,512_kbps_1mbps,1mbps_6mbps,6mbps_10mbps,10mbps_20mbps,20mbps_30mbps,30mbps_x,otras_velocidades,total_x,accesos_100_hogares,banda_ancha_fija,dial_up,total_y
4,2022,3,Chubut,109.0,1444.0,45707.0,30940.0,34682.0,15309.0,17563.0,20024.0,165778.0,84.38,164874,904,165778
28,2022,2,Chubut,109.0,1444.0,45804.0,31217.0,34290.0,14526.0,15394.0,20021.0,162805.0,83.19,161901,904,162805
52,2022,1,Chubut,113.0,1677.0,61369.0,31856.0,33080.0,13871.0,14055.0,15607.0,171628.0,88.05,170701,927,171628
76,2021,4,Chubut,126.0,1294.0,63698.0,39568.0,31175.0,5780.0,2904.0,15763.0,160308.0,82.57,159364,944,160308
100,2021,3,Chubut,104.0,105.0,89168.0,26281.0,18538.0,5320.0,1643.0,14907.0,156066.0,80.71,155266,800,156066
124,2021,2,Chubut,104.0,105.0,89639.0,22605.0,16604.0,3896.0,1112.0,5172.0,139237.0,72.29,138412,825,139237
148,2021,1,Chubut,106.0,109.0,86986.0,21401.0,15392.0,2844.0,1112.0,4390.0,132340.0,68.99,131515,825,13234
172,2020,4,Chubut,106.0,109.0,88157.0,21824.0,12046.0,2818.0,1112.0,4402.0,130574.0,68.35,129749,825,130574
196,2020,3,Chubut,158.0,109.0,87559.0,22114.0,12217.0,2818.0,1110.0,3942.0,130027.0,68.34,129202,825,130027
220,2020,2,Chubut,158.0,109.0,69578.0,22188.0,10537.0,1473.0,614.0,3999.0,108656.0,57.34,107831,825,108656


In [122]:
df_nacionales.to_csv('datos_ordenados/nacionales.csv',index=False)
df_provinciales.to_csv('datos_ordenados/provinciales.csv',index=False)
df_locales.to_csv('datos_ordenados/locales.csv',index=False)